<a href="https://colab.research.google.com/github/cacress/CSC442/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import pandas
import pandas as pd

# Read in the file and print out the DataFrame
hulu_data = pd.read_csv("https://raw.githubusercontent.com/cacress/CSC442/refs/heads/main/datasets/titles_hulu.csv")
netflix_data = pd.read_csv("https://raw.githubusercontent.com/cacress/CSC442/refs/heads/main/datasets/titles_netflix.csv")

In [4]:
# Concatentate datasets
combined_df = pd.concat([hulu_data, netflix_data])
combined_df.shape
combined_df.index

/n/n


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       5840, 5841, 5842, 5843, 5844, 5845, 5846, 5847, 5848, 5849],
      dtype='int64', length=8248)

In [5]:
# Create a list of the columns that are categorical and those that are integer and those that have string values

import numpy as np
numerical = []
categorical = []
string = []

combined_df['genres'] = combined_df['genres'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
combined_df['production_countries'] = combined_df['production_countries'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

for column in combined_df.columns:
# check if the column contains more than 10 unique values then it is not categorical
  if combined_df[column].dtype == np.int64 or combined_df[column].dtype == np.float64:
    numerical.append(column)

# second check for numerical values
#HINT: use numpy datatypes
  elif combined_df[column].nunique() > 100:
    string.append(column)
  elif combined_df[column].dtype == np.object_:
    categorical.append(column)

# otherwise it would be String values
# Categorical: Type, Genres, Age certification, Production country
  else:
    categorical.append(column)

print("Numerical columns: ", numerical)
print("Categorical columns: ", categorical)
print("String columns: ", string)

Numerical columns:  ['release_year', 'runtime', 'seasons', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score']
Categorical columns:  ['type', 'age_certification', 'genres', 'production_countries']
String columns:  ['id', 'title', 'description', 'imdb_id']


In [6]:
# Convert all the columns in the categorical list to be of the type category

for column in categorical:
  combined_df[column] = combined_df[column].astype('category')

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8248 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    8248 non-null   object  
 1   title                 8247 non-null   object  
 2   type                  8248 non-null   category
 3   description           8217 non-null   object  
 4   release_year          8248 non-null   int64   
 5   age_certification     4944 non-null   category
 6   runtime               8248 non-null   int64   
 7   genres                0 non-null      category
 8   production_countries  0 non-null      category
 9   seasons               3436 non-null   float64 
 10  imdb_id               7710 non-null   object  
 11  imdb_score            7600 non-null   float64 
 12  imdb_votes            7583 non-null   float64 
 13  tmdb_popularity       8107 non-null   float64 
 14  tmdb_score            7777 non-null   float64 
dtypes: catego

In [7]:
# Sanity check
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8248 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    8248 non-null   object  
 1   title                 8247 non-null   object  
 2   type                  8248 non-null   category
 3   description           8217 non-null   object  
 4   release_year          8248 non-null   int64   
 5   age_certification     4944 non-null   category
 6   runtime               8248 non-null   int64   
 7   genres                0 non-null      category
 8   production_countries  0 non-null      category
 9   seasons               3436 non-null   float64 
 10  imdb_id               7710 non-null   object  
 11  imdb_score            7600 non-null   float64 
 12  imdb_votes            7583 non-null   float64 
 13  tmdb_popularity       8107 non-null   float64 
 14  tmdb_score            7777 non-null   float64 
dtypes: catego

In [8]:
# check for duplicate rows using the duplicated().sum() functions - returns number of duplicate rows
# num_duplicates = combined_df.duplicated().sum()
# print("Number of duplicate rows:", num_duplicates)

combined_df.info()

# Check if there are duplicate IDs
duplicates_exist = combined_df['id'].duplicated().any()
print("\nAre there duplicate IDs?:", duplicates_exist)

# Find the unique duplicate values
duplicate_values = combined_df[combined_df['id'].duplicated()]['id'].unique()
print("\nDuplicate ID values:", duplicate_values)

# Extract duplicate rows
duplicate_rows_df = combined_df[combined_df['id'].duplicated(keep=False)]
print("\nRows with Duplicate IDs:\n", duplicate_rows_df)

# Remove duplicates, keeping the row with the highest 'runtime'
# deduplicated_df = combined_df.loc[combined_df.groupby('id')['runtime'].idxmax()]
deduplicated_df = (
    combined_df
    .sort_values(by='runtime', ascending=False)
    .drop_duplicates(subset=['id'], keep='first')
    .sort_values(by='id')  # optional re-sort if you like
    .reset_index(drop=True)
)

duplicates_exist = deduplicated_df['id'].duplicated().any()
print("\nAre there duplicate IDs?:", duplicates_exist)

# Reset index for cleanliness
# deduplicated_df = deduplicated_df.reset_index(drop=True)

# Print the cleaned DataFrame
print("\nDataFrame after removing duplicates while keeping the highest runtime:\n", deduplicated_df)

deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8248 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    8248 non-null   object  
 1   title                 8247 non-null   object  
 2   type                  8248 non-null   category
 3   description           8217 non-null   object  
 4   release_year          8248 non-null   int64   
 5   age_certification     4944 non-null   category
 6   runtime               8248 non-null   int64   
 7   genres                0 non-null      category
 8   production_countries  0 non-null      category
 9   seasons               3436 non-null   float64 
 10  imdb_id               7710 non-null   object  
 11  imdb_score            7600 non-null   float64 
 12  imdb_votes            7583 non-null   float64 
 13  tmdb_popularity       8107 non-null   float64 
 14  tmdb_score            7777 non-null   float64 
dtypes: catego

In [9]:
# Combine the imdb_score and tmdb_score columns into one, averaged column.
# Remove any rows that have neither imdb_score or tmdb_score

# Transform tmdb_score to match imdb_score's scale (1-10)
deduplicated_df["tmdb_score_scaled"] = deduplicated_df["tmdb_score"] * (9/10) + 1

# Compute the final combined score (average)
deduplicated_df["average_score"] = deduplicated_df[["tmdb_score_scaled", "imdb_score"]].mean(axis=1, skipna=True)

# Remove rows where both imdb_score and tmdb_score are NaN
# deduplicated_df = deduplicated_df.dropna(subset=['average_score']).reset_index(drop=True)

# Print final number of rows after filtering
print("Final number of rows after removing rows with no scores:", len(deduplicated_df))

deduplicated_df.info()

Final number of rows after removing rows with no scores: 8161
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    8161 non-null   object  
 1   title                 8160 non-null   object  
 2   type                  8161 non-null   category
 3   description           8130 non-null   object  
 4   release_year          8161 non-null   int64   
 5   age_certification     4864 non-null   category
 6   runtime               8161 non-null   int64   
 7   genres                0 non-null      category
 8   production_countries  0 non-null      category
 9   seasons               3370 non-null   float64 
 10  imdb_id               7625 non-null   object  
 11  imdb_score            7516 non-null   float64 
 12  imdb_votes            7499 non-null   float64 
 13  tmdb_popularity       8020 non-null   float64 

In [10]:
# Remove specified columns from the dataset using "drop()"
# ---- Ran the below line once ----
deduplicated_df = deduplicated_df.drop(columns=['description', 'id', 'imdb_score', 'tmdb_score', 'tmdb_score_scaled'])
deduplicated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8161 entries, 0 to 8160
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   title                 8160 non-null   object  
 1   type                  8161 non-null   category
 2   release_year          8161 non-null   int64   
 3   age_certification     4864 non-null   category
 4   runtime               8161 non-null   int64   
 5   genres                0 non-null      category
 6   production_countries  0 non-null      category
 7   seasons               3370 non-null   float64 
 8   imdb_id               7625 non-null   object  
 9   imdb_votes            7499 non-null   float64 
 10  tmdb_popularity       8020 non-null   float64 
 11  average_score         8039 non-null   float64 
dtypes: category(4), float64(4), int64(2), object(2)
memory usage: 542.8+ KB


In [ ]:
# Transform the genre column into a string type, with the value being the first element in the list of genres.

# Function to extract the first genre
def extract_first_genre(genre_list):
    if isinstance(genre_list, list) and len(genre_list) > 0:
        return genre_list[0]  # Get the first genre
    else:
        return pd.NA  # Assign pandas' NA value if the list is empty

# Apply function to the 'genres' column
copy_df = deduplicated_df.copy()

copy_df['genres_string'] = copy_df['genres'].apply(extract_first_genre)

deduplicated_df.info()
copy_df.info()

# Get all unique genre values as a list
print(copy_df.head())
